# 320 - M3 Experiment #1 - Hierarchical NER with Pero OCR dataset

Requirements : 
* Create datasets in `m2_joint-labelling_for_ner` : `M2_200-prepare_datasets_joint_labelling`

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

3


In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
#!pip install --upgrade transformers datasets spacy transformers[sentencepiece] seqeval

## Initialisation
Set the BASE path.
If run on Google Colab, will also mout Google Drive to the moutpoint given below

In [4]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  DATA_BASE = Path('/work/stual/res_ICDAR/method_2')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_3')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m3_hierarchical_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m3_hierarchical_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_3


## Constants

In [5]:
RUN_CAMEMBERT_IO = False
RUN_CAMEMBERT_IOB2 = False
#Can't run together because of convert_tokenizer_
RUN_PTRN_CAMEMBERT_IO = False
RUN_PTRN_CAMEMBERT_IOB2 = True

# Number of times a model will be trained & evaluated on each a dataset
N_RUNS = 5

## CamemBERT - Config

In [6]:
# COMMON CONSTANTS

TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100,
    "max_steps": 5000,
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 1e-5,
    "load_best_model_at_end": True,
    "greater_is_better":True,
    "metric_for_best_model": "f1",
    "save_strategy": "steps",
    "save_steps": 100,
    "save_total_limit": 1
}

In [7]:
import pandas as pd
from datasets import load_from_disk

#Print examples from datasets
def loadExample(INPUT_DIR,set_length:int,i:int,subset:str):
    set_ = load_from_disk(INPUT_DIR / f"huggingface_{set_length}")
    data = {"tokens": set_[subset][i]["tokens"],
            "labels": set_[subset][i]["ner_tags"]}
    df = pd.DataFrame.from_dict(data)
    print(df)

## Hierarchical NER : Trees

### Tree with IO Labels

In [8]:
from hierarchicalNER.trees import Ltree, Wtree
Ltree.pretty_print(unicodelines=True, nodedist=2)
print(Wtree)

                                                             EN                                                                                        
 ┌──────────────┬───────────────────────┬────────────────────┼────────────────────────────────────────┬───────────────────────────────────────────┐      
 O            I-PER                   I-ACT                I-DESC                                   I-SPAT                                     I-TITRE 
 │      ┌───────┴──────────┐            │        ┌───────────┼───────────────┐            ┌───────────┼───────────────┬───────────────┐           │      
O+O  I-PER+O         I-PER+i_TITREH  I-ACT+O  I-DESC+O  I-DESC+i_ACT  I-DESC+i_TITREP  I-SPAT+O  I-SPAT+i_LOC  I-SPAT+i_CARDINA  I-SPAT+i_FT  I-TITRE+O
                                                                                                                      L                                

(-100.0
  (1.0 1.0)
  (1.0 1.0 1.0)
  (1.0 1.0)
  (1.0 1.0 1.0 1.0)
  (1.0 1.0 1.0 

### Tree with IOB2 Labels

## 321 - Train & eval : IO Ref dataset with CamemBERT model

In [9]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "io"
FOLDER = "321-camembert-ner-hierarchical-loss-io"

### 321.1 Load IO dataset

In [10]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-320-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_io_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-320-experiment_2_metrics'))

In [11]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens             labels
0  Duffau-Pauillac            I-PER+O
1                (            I-PER+O
2              Chs            I-PER+O
3                )            I-PER+O
4                ,                O+O
5          Enghien       I-SPAT+i_LOC
6                ,           I-SPAT+O
7               16  I-SPAT+i_CARDINAL
8                .                O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 321.2 Fine-tuning with IO labels - train & eval

In [12]:
from config import logger
from datasets import load_from_disk
import json
from hierarchicalNER.util_IO import init_model, train_eval_loop

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

2023-02-03 09:34:56.291847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 09:34:57.337400: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-03 09:34:57.337513: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-03 09:34:57.337523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Tree positions of all the leaves{'O+O': (0, 0), 'I-PER+O': (1, 0), 'I-PER+i_TITREH': (1, 1), 'I-ACT+O': (2, 0), 'I-DESC+O': (3, 0), 'I-DESC+i_ACT': (3, 1), 'I-DESC+i_TITREP': (3, 2), 'I-SPAT+O': (4, 0), 'I-SPAT+i_LOC': (4, 1), 'I-SPAT+i_CARDINAL': (4, 2), 'I-SPAT+i_FT': (4, 3), 'I-TITRE+O': (5, 0)} #1
Num of classes : 12


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
import time
import datetime
from hierarchicalNER.util_IO import _convert_tokenizer

if RUN_CAMEMBERT_IO:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

Skipped finetuning model for IO labels


## 322 - Train & eval : IOB2 Ref dataset with CamemBERT model

In [14]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "iob2"
FOLDER = "322-camembert-ner-hierarchical-loss-iob2"

### 322.1 Load IOB2 dataset

In [15]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-320-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_iob2_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-320-experiment_2_metrics'))

In [16]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens               labels
0  Duffau-Pauillac            I-b_PER+O
1                (            I-i_PER+O
2              Chs            I-i_PER+O
3                )            I-i_PER+O
4                ,                  O+O
5          Enghien       I-b_SPAT+b_LOC
6                ,           I-i_SPAT+O
7               16  I-i_SPAT+b_CARDINAL
8                .                  O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 322.2 Fine-tuning with IOB2 labels - train & eval

In [17]:
from config import logger
from datasets import load_from_disk
from hierarchicalNER.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer
import json

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

Tree positions of all the leaves{'O+O': (0, 0, 0), 'I-b_PER+O': (1, 0, 0), 'I-i_PER+O': (1, 0, 1), 'I-b_PER+b_TITREH': (1, 1, 0), 'I-i_PER+b_TITREH': (1, 1, 1), 'I-i_PER+i_TITREH': (1, 1, 2), 'I-b_ACT+O': (2, 0, 0), 'I-i_ACT+O': (2, 0, 1), 'I-b_DESC+O': (3, 0, 0), 'I-i_DESC+O': (3, 0, 1), 'I-b_DESC+b_ACT': (3, 1, 0), 'I-i_DESC+b_ACT': (3, 1, 1), 'I-i_DESC+i_ACT': (3, 1, 2), 'I-b_DESC+b_TITREP': (3, 2, 0), 'I-i_DESC+b_TITREP': (3, 2, 1), 'I-i_DESC+i_TITREP': (3, 2, 2), 'I-b_SPAT+O': (4, 0, 0), 'I-i_SPAT+O': (4, 0, 1), 'I-b_SPAT+b_LOC': (4, 1, 0), 'I-i_SPAT+b_LOC': (4, 1, 1), 'I-i_SPAT+i_LOC': (4, 1, 2), 'I-b_SPAT+b_CARDINAL': (4, 2, 0), 'I-i_SPAT+b_CARDINAL': (4, 2, 1), 'I-i_SPAT+i_CARDINAL': (4, 2, 2), 'I-b_SPAT+b_FT': (4, 3, 0), 'I-i_SPAT+b_FT': (4, 3, 1), 'I-i_SPAT+i_FT': (4, 3, 2), 'I-b_TITRE+O': (5, 0, 0), 'I-i_TITRE+O': (5, 0, 1)} #1
Num of classes : 29


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import time
import datetime

if RUN_CAMEMBERT_IOB2:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
else:
    print("Skipped finetuning model for IOB2 labels")

Skipped finetuning model for IOB2 labels


## 323 - Train & eval : IO Ref dataset with Pretrained CamemBERT model

In [19]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "io"
FOLDER = "323-pretrained-camembert-ner-hierarchical-loss-io"

### 323.1 Load IO dataset

In [20]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-320-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_io_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-320-experiment_2_metrics'))

In [21]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens             labels
0  Duffau-Pauillac            I-PER+O
1                (            I-PER+O
2              Chs            I-PER+O
3                )            I-PER+O
4                ,                O+O
5          Enghien       I-SPAT+i_LOC
6                ,           I-SPAT+O
7               16  I-SPAT+i_CARDINAL
8                .                O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 323.2 Fine-tuning with IO labels - train & eval

In [22]:
from config import logger
from datasets import load_from_disk
from hierarchicalNER.util_IO import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [23]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IO:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning pretrained model for IO labels")

Skipped finetuning pretrained model for IO labels


## 324 - Train & eval : IOB2 Ref dataset with Pretrained CamemBERT model

In [24]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "iob2"
FOLDER = "324-pretrained-camembert-ner-hierarchical-loss-iob2"

### 324.1 Load IOB2 dataset

In [25]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = DATA_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m3-320-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_iob2_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_3/m3-320-experiment_2_metrics'))

In [26]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens               labels
0  Duffau-Pauillac            I-b_PER+O
1                (            I-i_PER+O
2              Chs            I-i_PER+O
3                )            I-i_PER+O
4                ,                  O+O
5          Enghien       I-b_SPAT+b_LOC
6                ,           I-i_SPAT+O
7               16  I-i_SPAT+b_CARDINAL
8                .                  O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [27]:
from config import logger
from datasets import load_from_disk
from hierarchicalNER.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer
import json

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [28]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IOB2:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning pretrained model for IOB2 labels")

Some weights of the model checkpoint at HueyNemud/das22-10-camembert_pretrained were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at HueyNemud/das22-10-camembert_pretrained and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this mo

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Precision-l1,Recall-l1,F1-l1,Accuracy-l1,Precision-l2,Recall-l2,F1-l2,Accuracy-l2,Precision-das,Recall-das,F1-das,Accuracy-das,Precision-l1l2,Recall-l1l2,F1-l1l2,Accuracy-l1l2,Precision-all,Recall-all,F1-all,Accuracy-all,Per,Act,Act L1,Act L2,Desc,Titreh,Titrep,Spat,Loc,Cardinal,Ft,Titre
100,No log,0.663207,0.940189,0.924033,0.932041,0.930904,0.922798,0.934908,0.928814,0.944824,0.939083,0.913449,0.926088,0.963554,0.929693,0.925885,0.927785,0.949127,0.931891,0.926475,0.929175,0.931410,0.929814,0.925448,0.927626,0.954189,"{'precision': 0.9484536082474226, 'recall': 0.9526627218934911, 'f1': 0.9505535055350554, 'number': 676}","{'precision': 0.8956043956043956, 'recall': 0.9421965317919075, 'f1': 0.9183098591549297, 'number': 519}","{'precision': 0.8956043956043956, 'recall': 0.9721669980119284, 'f1': 0.9323164918970448, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.9191489361702128, 'recall': 0.9337175792507204, 'f1': 0.9263759828448892, 'number': 694}","{'precision': 0.924380704041721, 'recall': 0.9304461942257218, 'f1': 0.9274035317200785, 'number': 762}","{'precision': 0.9553314121037464, 'recall': 0.9778761061946902, 'f1': 0.9664723032069971, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
200,No log,0.385018,0.943787,0.937599,0.940683,0.936851,0.918168,0.936483,0.927235,0.945457,0.942818,0.922104,0.932346,0.962541,0.940037,0.940037,0.940037,0.955454,0.939085,0.936356,0.937718,0.937358,0.928781,0.930144,0.929462,0.953999,"{'precision': 0.9384164222873901, 'recall': 0.9467455621301775, 'f1': 0.9425625920471281, 'number': 676}","{'precision': 0.8923357664233577, 'recall': 0.9421965317919075, 'f1': 0.9165885660731022, 'number': 519}","{'precision': 0.8923357664233577, 'recall': 0.9721669980119284, 'f1': 0.9305423406279734, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.9199438202247191, 'recall': 0.9438040345821326, 'f1': 0.9317211948790898, 'number': 694}","{'precision': 0.926923076923077, 'recall': 0.9488188976377953, 'f1': 0.9377431906614787, 'number': 762}","{'precision': 0.9608127721335269, 'recall': 0.976401179941003, 'f1': 0.9685442574981712, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
300,No log,0.289530,0.951657,0.947999,0.949824,0.945077,0.923547,0.951181,0.937161,0.954948,0.931534,0.942077,0.936776,0.971906,0.955523,0.960149,0.957830,0.967223,0.938378,0.951468,0.944877,0.945457,0.927032,0.947168,0.936992,0.963427,"{'precision': 0.9502196193265008, 'recall': 0.9600591715976331, 'f1': 0.9551140544518028, 'number': 676}","{'precision': 0.9297912713472486, 'recall': 0.9441233140655106, 'f1': 0.936902485659656, 'number': 519}","{'precision': 0.9297912713472486, 'recall': 0.974155069582505, 'f1': 0.9514563106796117, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.18181818181818182, 'recall': 0.26666666666666666, 'f1': 0.21621621621621623, 'number': 30}","{'precision': 0.32608695652173914, 'recall': 0.45454545454545453, 'f1': 0.379746835443038, 'number': 33}","{'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1': 0.3333333333333333, 'number': 6}","{'precision': 0.9392655367231638, 'recall': 0.9582132564841499, 'f1': 0.9486447931526392, 'number': 694}","{'precision': 0.9392764857881137, 'recall': 0.9540682414698163, 'f1': 0.9466145833333334, 'number': 762}","{'precision': 0.

The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
/lrde/home2/stual/stage_DAS/m3_hierarchical_ner/hierarchicalNER/util_IOB2.py:205: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precisio

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/li

Trainer is attempting to log a value of "{'precision': 0.9297912713472486, 'recall': 0.974155069582505, 'f1': 0.9514563106796117, 'number': 503}" of type <class 'dict'> for key "eval/ACT_L1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}" of type <class 'dict'> for key "eval/ACT_L2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.18181818181818182, 'recall': 0.26666666666666666, 'f1': 0.21621621621621623, 'number': 30}" of type <class 'dict'> for key "eval/DESC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.32608695652173914, 'recall': 0.45454545454545453, 'f1': 0.379746835443038, 'number': 33

...

Trainer is attempting to log a value of "{'precision': 0.75, 'recall': 0.42857142857142855, 'f1': 0.5454545454545454, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /work/stual/res_ICDAR/method_3/tmp/324-pretrained-camembert-ner-hierarchical-loss-iob2/checkpoint-1300
Configuration saved in /work/stual/res_ICDAR/method_3/tmp/324-pretrained-camembert-ner-hierarchical-loss-iob2/checkpoint-1300/config.json
Model weights saved in /work/stual/res_ICDAR/method_3/tmp/324-pretrained-camembert-ner-hierarchical-loss-iob2/checkpoint-1300/pytorch_model.bin
tokenizer config file saved in /work/stual/res_ICDAR/

The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1685
  Batch size = 16


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Trainer is attempting to log a value of "{'precision': 0.9407971864009379, 'recall': 0.9525222551928784, 'f1': 0.946623414921852, 'number': 1685}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorre

Trainer is attempting to log a value of "{'precision': 0.953185955786736, 'recall': 0.9619422572178478, 'f1': 0.9575440888308295, 'number': 762}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9824046920821115, 'recall': 0.9882005899705014, 'f1': 0.9852941176470589, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.4, 'recall': 0.2857142857142857, 'f1': 0.3333333333333333, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict

Run-time is equal to 0:08:59.110869
